In [ ]:
# =========================================================
# Hệ Thống Nhận Diện Khay Cơm (Cắt Theo Layout Cố Định)
# =========================================================
# !pip install -q gradio==4.* pillow numpy tensorflow

# Bỏ import cv2 vì không còn dùng đến
import numpy as np
from PIL import Image
import gradio as gr
import tensorflow as tf
import random

# ----------------- CONFIG (BẠN CẦN THAY ĐỔI) -----------------
MODEL_PATH  = "/content/sample_data.h5" # <— THAY ĐỔI ĐƯỜNG DẪN TỚI MODEL CỦA BẠN

CLASS_NAMES = [
    '1. Cơm trắng - 10.000', '10. Rau xào - 10000', '11. Trứng chiên - 25000', '12. Canh bí đao - 12000', '13. Canh bí đỏ - 12000', '14. Dưa leo - 5000', '15. Lạp sưởng - 15000', '16. Nước chấm - 3000', '17. Khay trống - 0', '2. Đậu hũ sốt cà - 25000', '3. Cá hú kho - 30000', '4. Thịt kho trứng - 30000', '5. Thịt kho - 25000', '6. Canh chua cá - 25000', '7. Canh chua - 10000', '8. Sường nướng - 30000', '9. Canh rau - 7000'
]

# --- BỐ CỤC CẮT ẢNH CỐ ĐỊNH ---
# Tọa độ tương đối (x1, y1, x2, y2) của 5 ô trên khay.
# (0,0) là góc trên cùng bên trái, (1,1) là góc dưới cùng bên phải.
# BẠN CÓ THỂ TINH CHỈNH CÁC TỌA ĐỘ NÀY NẾU KẾT QUẢ CẮT BỊ LỆCH.
LAYOUT_KHAY5 = {
    "Ô 1": (0.00, 0.00, 0.58, 0.55),  # canh (trên trái)
    "Ô 2": (0.60, 0.00, 1.00, 0.55),  # cơm (trên phải)
    "Ô 3": (0.00, 0.56, 0.32, 1.00),  # dưới trái
    "Ô 4": (0.33, 0.56, 0.66, 1.00),  # giữa dưới
    "Ô 5": (0.67, 0.56, 1.00, 1.00),  # dưới phải
}

APP_NAME = "Hệ Thống Nhận Diện Khay Cơm"
BASE_CSS = """
/* Giữ nguyên CSS từ phiên bản trước */
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&family=Outfit:wght@700;800&display=swap');
.gradio-container { max-width: 1024px !important; }
.header {
  background: linear-gradient(135deg, #43A047, #1E88E5); color: white;
  padding: 22px 20px; border-radius: 18px; margin-bottom: 14px;
}
.header h1 { font-family: 'Outfit', sans-serif; font-weight:800; font-size: 30px;}
.header p  { margin:0; font-size:15px; opacity:.95;}
"""
# ----------------------------------------------------------------

# Tải model và lấy kích thước input
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    input_h, input_w, input_c = model.input_shape[1:4]
    print("✅ Model đã được tải thành công!")
except Exception as e:
    print(f"🛑 Lỗi không thể tải model: {e}")
    model = None

# --- CÁC HÀM TIỆN ÍCH (Giữ nguyên) ---
def _to_rgb_or_gray(pil_img, want_c):
    return pil_img.convert("L") if want_c == 1 else pil_img.convert("RGB")

def preprocess(pil_img):
    img = _to_rgb_or_gray(pil_img, input_c)
    img = img.resize((input_w, input_h), Image.BILINEAR)
    arr = np.array(img, dtype=np.float32) / 255.0
    if input_c == 1: arr = arr[..., np.newaxis]
    arr = arr[np.newaxis, ...]
    return arr

def _to_probs(vec):
    v = np.asarray(vec, dtype=np.float32).ravel()
    if np.any(v < 0) or v.sum() <= 0.99 or v.sum() >= 1.01:
        e = np.exp(v - v.max())
        v = e / (e.sum() + 1e-8)
    return v

# --- HÀM BÓC TÁCH KHAY ĐỒ ĂN (Phiên bản Layout Cố Định) ---
def segment_tray_by_layout(pil_image):
    """
    Cắt ảnh khay đồ ăn dựa trên một layout cố định.
    """
    cropped_images = []
    img_width, img_height = pil_image.size

    # Sắp xếp các ô theo key ("Ô 1", "Ô 2",...) để đảm bảo thứ tự
    sorted_layout_keys = sorted(LAYOUT_KHAY5.keys())

    for key in sorted_layout_keys:
        # Lấy tọa độ tương đối (0->1)
        rel_coords = LAYOUT_KHAY5[key]

        # Chuyển đổi sang tọa độ pixel tuyệt đối
        abs_x1 = int(rel_coords[0] * img_width)
        abs_y1 = int(rel_coords[1] * img_height)
        abs_x2 = int(rel_coords[2] * img_width)
        abs_y2 = int(rel_coords[3] * img_height)

        # Cắt ảnh theo tọa độ pixel
        cropped_img = pil_image.crop((abs_x1, abs_y1, abs_x2, abs_y2))
        cropped_images.append(cropped_img)

    return cropped_images

# --- HÀM ĐIỀU PHỐI DỰ ĐOÁN (Cập nhật để gọi hàm mới) ---
def predict_tray(tray_image):
    if tray_image is None:
        return "Vui lòng tải ảnh khay đồ ăn.", None
    if model is None:
        return "Lỗi: Model chưa được tải. Vui lòng kiểm tra lại cấu hình.", None

    # Gọi hàm cắt ảnh theo layout cố định
    list_of_food_images = segment_tray_by_layout(tray_image)

    predictions = []
    # Lấy tên các ô đã sắp xếp để hiển thị
    sorted_keys = sorted(LAYOUT_KHAY5.keys())

    for i, food_img in enumerate(list_of_food_images):
        arr = preprocess(food_img)
        raw = model.predict(arr, verbose=0)[0]
        probs = _to_probs(raw)

        idx = np.argmax(probs)
        pred_class = CLASS_NAMES[idx]
        conf = probs[idx]

        # Thêm tên ô (Ô 1, Ô 2,...) vào kết quả
        predictions.append(f"- **{sorted_keys[i]}**: {pred_class} (Độ tin cậy: {conf:.1%})")

    result_markdown = "### 🍱 Kết Quả Nhận Diện Khay Đồ Ăn\n" + "\n".join(predictions)
    return result_markdown, list_of_food_images

# --- GIAO DIỆN (UI - Giữ nguyên) ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="green"), css=BASE_CSS) as demo:
    gr.HTML(f"""
    <div class="header">
      <h1>{APP_NAME}</h1>
      <p>Tải ảnh một khay đồ ăn, hệ thống sẽ tự động bóc tách và nhận diện 5 món ăn có trong khay.</p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            inp = gr.Image(type="pil", label="Tải ảnh khay đồ ăn")
            btn = gr.Button("✨ Phân Tích Khay Đồ Ăn", variant="primary")

        with gr.Column(scale=1):
            out_text = gr.Markdown("Kết quả sẽ hiển thị ở đây...")

    out_gallery = gr.Gallery(label="Các món ăn đã được bóc tách",
                             columns=5, rows=1, object_fit="contain", height="auto")

    btn.click(fn=predict_tray, inputs=[inp], outputs=[out_text, out_gallery])

# Chạy ứng dụng
demo.launch(share=True, show_error=True)